Summpip Paper Implementation

In [27]:
#make all necessary imports
import numpy as np
import nltk
import nltk.data
import os
import spacy
import tensorflow_hub as hub
import copy
import gensim.downloader as api
!pip install rouge
import spacy 
from tqdm import tqdm
from rouge import Rouge
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import wordnet as wn
from ordered_set import OrderedSet
sentence_detector = nltk.data.load('tokenizers/punkt/english.pickle')
use_tensorflow_embedding = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
glove_word_vectors = api.load("glove-wiki-gigaword-100")

[nltk_data] Downloading package wordnet to /home/siddhant/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/siddhant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[==================================================] 100.0% 128.1/128.1MB downloaded


Extracting Sentences out of Data. (Preprocessing Step)

In [28]:
with open('./summarizer/DocsData.txt', 'r') as file:
    lines = file.readlines()

doc_separator_tag="story_separator_special_tag"
cnt = 0
line_sent_list = []
for line in lines:
    line = line.replace(doc_separator_tag, "")
    sent_list = sentence_detector.tokenize(line.strip())
    line_sent_list.append(sent_list)
    cnt += 1
    if cnt <= 5:
        print(sent_list)

docs = line_sent_list[:1]

["GOP Eyes Gains As Voters In 11 States Pick Governors Enlarge this image toggle caption Jim Cole / AP Jim Cole / AP Voters in 11 states will pick their governors tonight , and Republicans appear on track to increase their numbers by at least one , with the potential to extend their hold to more than two - thirds of the nation 's top state offices .", 'Eight of the gubernatorial seats up for grabs are now held by Democrats ; three are in Republican hands .', "Republicans currently hold 29 governorships , Democrats have 20 , and Rhode Island 's Gov.", 'Lincoln Chafee is an Independent .', "Polls and race analysts suggest that only three of tonight 's contests are considered competitive , all in states where incumbent Democratic governors are n't running again : Montana , New Hampshire and Washington .", 'While those state races remain too close to call , Republicans are expected to wrest the North Carolina governorship from Democratic control , and to easily win GOP - held seats in Utah

Graph Construction Step (Node will be a sentence)

In [38]:
#using spacynlp for POS(part of speech) tagging
spacynlp=spacy.load("en_core_web_sm")
verbs_to_escape = ["be", "is","am","are","was", "were", "being","been","do","did",
               "done","have","had","get","got","gotten"]

discourseMarkers=["for","so","because","since","therefore","consequently","additionally","furthermore","moreover",
         "but","however","although","despite","similarly","otherwise","whereas","while","unlike","thus",
        "instead","nevertheless","afterward","finally","subsequently","conversely","later","next","then",
         "likewise","compared","besides","further","as","also","equally","hence","accordingly","stil",
        "simultaneously"]
def convert_verb_to_nouns(sentence):
    tokens = spacynlp(sentence)
    nouns_arr = []
    for token in tokens:
        if token.pos_ == 'VERB' and token.text not in verbs_to_escape:
            tokenNouns = getNouns(token.text)
            # print('Here ', token.text, '-->> ', tokenNouns)
            nouns_arr.extend(tokenNouns)
            # print(nouns_arr)
    return nouns_arr

#Using getNouns to get lemmas(base word) of verbs
def getNouns(verb):
    baseform = wn.morphy(verb, wn.VERB)
    if baseform:
        lemmas = wn.lemmas(baseform, pos='v')
        noun_forms = []
        for lemma in lemmas:
            nouns = [form.name() for form in lemma.derivationally_related_forms()]
            noun_forms.extend(nouns)
        noun_forms = OrderedSet(noun_forms)
        return noun_forms
    else:
        return []
    
#find similar nouns based on cosine similarity from glove vectors dataset
def find_similar_nouns(nouns_list, word_vecs, thres=0.75):
    similar_nouns = []
    nouns = list(set(nouns_list))
    for noun in nouns:
        try:
            sim_word_list = word_vecs.most_similar(positive=[noun])
            sim_word_list = [tup[0] for tup in sim_word_list if tup[1]>thres]
            similar_nouns.extend(sim_word_list)
        except KeyError:
            pass
        similar_nouns.extend(nouns_list)
        return list(set(similar_nouns))
    
#Check for deverbal nouns in successive sentence
def check_deverbal_noun(similar_nouns, sent2):
    edge = False
    sent2words = spacynlp(sent2)
    if len(sent2words)>0:
        for token in sent2words:
            if token.pos_ == 'NOUN':
                if token.text in similar_nouns:
                    edge = True
                    break
    return edge

def check_discourse_markers(sent1, sent2):
    edge = False
    sent2words = spacynlp(sent2)
    if len(sent2words)>0:
        firstWord = sent2words[0].text
        if firstWord.lower() in discourseMarkers:
            edge = True
    return edge

In [43]:
embed_size = 512 #for tensorflow embedding
adj_matrix = None
sentence_embeddings_matrix = None

#declares adjacency matrix and gives number of nodes and sentence embeddings(vector of size 512, per sentence)
def build_sentence_graph(document):
    num_nodes = len(document)
    print(document)
    print('Number of nodes: ', num_nodes)
    sentence_embeddings_matrix = np.zeros([num_nodes, embed_size])
    adj_matrix = np.zeros([num_nodes, num_nodes])
    cnt = 0
    for sentence in document:
        sentence_embeddings_matrix[cnt] = use_tensorflow_embedding([sentence]).numpy()[0]
        cnt += 1
    return  num_nodes, adj_matrix,sentence_embeddings_matrix



#Fills Adjacency Matrix
def connect_sentence_nodes(document, num_nodes, adj_matrix, sentence_embeddings_matrix):
    for i in range(num_nodes):
        connect = False
        sentence1 = document[i]
        for j in range(i+1, num_nodes):
            sentence2 = document[j]
            if j-i == 1:
                nouns_list = convert_verb_to_nouns(sentence1)
                similar_nouns_list = find_similar_nouns(nouns_list, glove_word_vectors, 0.5)
                print('Similar Nouns list: ', similar_nouns_list, ' for sentence ', sentence1)
                if similar_nouns_list is not None:
                    connect = check_deverbal_noun(similar_nouns_list, sentence2)
                if connect == False:
                    connect = check_discourse_markers(sentence1, sentence2)
                if connect == True:
                    adj_matrix[i,j] = 1
                    adj_matrix[j,i] = 1
    print(adj_matrix)

for doc in docs:
    num_nodes, adj_matrix, sentence_embeddings_matrix = build_sentence_graph(doc)
    connect_sentence_nodes(doc, num_nodes, adj_matrix, sentence_embeddings_matrix)


["GOP Eyes Gains As Voters In 11 States Pick Governors Enlarge this image toggle caption Jim Cole / AP Jim Cole / AP Voters in 11 states will pick their governors tonight , and Republicans appear on track to increase their numbers by at least one , with the potential to extend their hold to more than two - thirds of the nation 's top state offices .", 'Eight of the gubernatorial seats up for grabs are now held by Democrats ; three are in Republican hands .', "Republicans currently hold 29 governorships , Democrats have 20 , and Rhode Island 's Gov.", 'Lincoln Chafee is an Independent .', "Polls and race analysts suggest that only three of tonight 's contests are considered competitive , all in states where incumbent Democratic governors are n't running again : Montana , New Hampshire and Washington .", 'While those state races remain too close to call , Republicans are expected to wrest the North Carolina governorship from Democratic control , and to easily win GOP - held seats in Utah